# Lecture 4: Password CPA Attack - Attack

In this example we want to improve the password check again to be resistant against the attack from the last tutorial.

## Improving the code

Let's first recap the password checking loop from the last lecture:
```c
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    if (stored_password[i] != passwd[i])
    {
        password_wrong = 1;
    }
}
```

The differences attack discussed in the last example worked because of the different power consumption when executing the code inside the if clause. This is addressed by the following code.

```c
uint8_t password_wrong = 0;
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    password_wrong |= stored_password[i] ^ passwd[i];
}
```

This is an excerpt from `4_password_fixed.c`.

In [ ]:
import securec
from securec import util
scope, target = util.init()

In [ ]:
securec.util.compile_and_flash('./4_password_fixed.c')

In [ ]:
import struct
import time
import warnings
import numpy as np

scope.default_setup()
scope.adc.samples = 500

def capture(attempt, count=1):
    if isinstance(attempt, str):
        attempt = attempt.encode('iso-8859-1')
    elif isinstance(attempt, int):
        attempt = bytes([attempt])
    traces = []
    for _ in range(count):
        scope.arm()
        target.simpleserial_write('p', attempt + b'\x00' * (10 - len(attempt)))
        result = target.simpleserial_read('p', 1)
        traces.append(util.capture())
    return np.mean(np.array(traces), axis=0), not bool(result[0])

In [ ]:
import math
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10, Turbo256, Inferno256
from bokeh.models import Span, Label, BoxAnnotation
from bokeh.layouts import column, row

output_notebook()

## Preparation

In [ ]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

In [ ]:
import random
import tqdm
import tqdm.notebook

traces = []
for _ in tqdm.notebook.tqdm(range(500)):
    attempt = random.randint(0, 255)
    traces.append((attempt, capture(attempt)[0]))

## Pearson correlation coefficient
An interesting statistical formula to face this problem is given by the *Pearson correlation coefficient*. For two random variables $X, Y$ it is defined as

$$\rho_{X,Y} := \frac{\mathrm{Cov}(X, Y)}{\sqrt{\mathrm{Var}(X)} \sqrt{\mathrm{Var}(Y)}} \ \in [-1, 1]\,.$$

For two samples of finite length $x = {x_1, ..., x_n}$, $y = {y_1, ..., y_n}$ it can be defined as 

$$r_{x,y} := \frac{\sum_{i=1}^n (x_i - \bar x)(y_i - \bar y)}{\sqrt{\sum_{i=1}^n (x_i - \bar x)^2}\sqrt{\sum_{i=1}^n (y_i - \bar y)^2}} \ \in [-1, 1]\,,$$

where $\bar x := \frac{1}{n} \sum_{i=1}^n x_i$ is the mean of a sample $x$.

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 1</div>
    
1. Implement Pearson correlation coefficient in python.
2. Test your implementation against the following (x, y)-data with `size = 50`:
    1. `(range(size), 5 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size))` => $r \approx 0.99$
    2. `(range(size), np.array(range(size)) + np.random.uniform(-size, size, size=size) + 10)` => $r \approx 0.32$
    3. `(range(size), -3 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size) + 200)` => $r \approx -0.98$
    4. `(range(size), 100 * np.sin(np.array(range(size)) / size * np.pi + np.pi) + np.random.uniform(-size/10, size/10, size=size))` => $r \approx -0.05$
3. Visualize above example data.
</div>

## Attack

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 2</div>
    
Develop an attack using Pearson correlation coefficient.

</div>